In [1]:
import torch
from torch import nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np

# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # CIFAR10的均值和标准差
])

# 加载CIFAR10数据集
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 定义Simplified VGG16模型
class SimplifiedVGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(SimplifiedVGG16, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.maxpool(x)
        x = nn.functional.relu(self.conv2(x))
        x = self.maxpool(x)
        x = torch.flatten(x, 1)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 实例化两个VGG16模型
model_selector = SimplifiedVGG16()  # 用于选择最有价值5%测试用例的模型
model_predictor = SimplifiedVGG16()  # 用于对5%测试用例进行预测的模型

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_selector = model_selector.to(device)
model_predictor = model_predictor.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer_selector = optim.Adam(model_selector.parameters(), lr=0.001)  # 选择模型的优化器
optimizer_predictor = optim.Adam(model_predictor.parameters(), lr=0.001)  # 预测模型的优化器

# 训练模型
def train_model(model, dataloader, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# 训练选择模型
train_model(model_selector, train_loader, optimizer_selector)  # 训练选择模型

# 训练预测模型
train_model(model_predictor, train_loader, optimizer_predictor)  # 训练预测模型在训练集上
train_model(model_predictor, test_loader, optimizer_predictor)  # 训练预测模型在测试集上

# 评估模型并计算置信度
model_selector.eval()
uncertainties = []
labels = []
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        output = model_selector(x)  # 使用选择模型进行评估
        probabilities = torch.softmax(output, dim=1)
        max_prob, _ = torch.max(probabilities, dim=1)
        uncertainties.extend(max_prob.cpu().numpy())
        labels.extend(y.cpu().numpy())

# 选择最不自信的5%测试用例
threshold_index = int(len(uncertainties) * 0.05)
threshold = np.partition(uncertainties, -threshold_index)[threshold_index]
selected_indices = np.where(uncertainties < threshold)[0]

# 使用预测模型对挑选出的测试用例进行预测
correct_count = 0
total_count = 0
with torch.no_grad():
    for idx in selected_indices:
        img, label = test_dataset[idx]  # 获取图片和标签
        img = img.unsqueeze(0).to(device)  # 增加批次维度并移动到设备
        output = model_predictor(img)  # 使用预测模型进行预测
        _, predicted = torch.max(output, 1)
        if predicted.item() == label:
            correct_count += 1
        total_count += 1

# 检查 total_count 是否为 0
if total_count > 0:
    # 计算正确分类率
    accuracy = correct_count / total_count
    print(f"正确分类率: {accuracy:.4f}")
else:
    print("没有挑选出的测试用例，无法计算正确分类率。")

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/5], Loss: 1.1023
Epoch [2/5], Loss: 1.0652
Epoch [3/5], Loss: 0.6925
Epoch [4/5], Loss: 0.4153
Epoch [5/5], Loss: 0.2732
Epoch [1/5], Loss: 0.8462
Epoch [2/5], Loss: 0.5054
Epoch [3/5], Loss: 0.3725
Epoch [4/5], Loss: 0.6289
Epoch [5/5], Loss: 0.3664
Epoch [1/5], Loss: 1.0945
Epoch [2/5], Loss: 0.1916
Epoch [3/5], Loss: 0.0347
Epoch [4/5], Loss: 0.0624
Epoch [5/5], Loss: 0.0027
正确分类率: 0.8967
